# BBOB Vizier

## Generate data
Everything in root (not in ´notebooks´)

```bash
python -m carps.run +optimizer/randomsearch=config '+problem/BBOBVizier=glob(*)' 'seed=range(1,21)' +cluster=noctua -m  
python -m carps.run +optimizer/hebo=config '+problem/BBOBVizier=glob(*)' 'seed=range(1,21)' +cluster=noctua -m  
python -m carps.run +optimizer/smac20=blackbox '+problem/BBOBVizier=glob(*)' 'seed=range(1,21)' +cluster=noctua -m  
```

## Gather data
```bash
python -m carps.analysis.gather_data runs/RandomSearch/BBOBVizier
python -m carps.analysis.gather_data runs/HEBO/BBOBVizier
python -m carps.analysis.gather_data runs/SMAC3-BlackBoxFacade/BBOBVizier
```

In [ ]:
import pandas as pd
from pathlib import Path
from carps.analysis.gather_data import normalize_logs, get_interpolated_performance_df, load_logs, process_logs, load_set

rundirs = [
    "../runs/RandomSearch/BBOBVizier",
    "../runs/HEBO/BBOBVizier",
    "../runs/SMAC3-BlackBoxFacade/BBOBVizier",
]
scenario = "BBOBVizier"
figpath = Path(f"figures/{scenario}/")
figpath.mkdir(parents=True, exist_ok=True)


df, df_Cfg = load_set(paths=rundirs)
print(df.columns)
df = normalize_logs(df)
perf = get_interpolated_performance_df(df)
perf_time = get_interpolated_performance_df(df, x_column="time_norm")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from carps.analysis.utils import setup_seaborn
import seaborn as sns
import importlib
import carps
import carps.analysis
import carps.analysis.plot_ranking
importlib.reload(carps)
importlib.reload(carps.analysis)
importlib.reload(carps.analysis.plot_ranking)
from carps.analysis.plot_ranking import plot_ranking
from carps.analysis.performance_over_time import plot_performance_over_time, setup_seaborn, savefig, get_color_palette
import seaborn as sns
import matplotlib

perf_col = "trial_value__cost_inc_norm"
plot_ranking(gdf=df, scenario=scenario, set_id="unknown", perf_col=perf_col)

lineplot_kwargs = dict(linewidth=3)
fig, ax = plot_performance_over_time(
    df=df, 
    x="n_trials",  # we have 100 trials everywhere, if n trials would differ, use n_trials_norm 
    y="trial_value__cost_inc_norm", 
    hue="optimizer_id", 
    figure_filename=figpath / "performance_over_time_trials.pdf", 
    figsize=(6,4), 
    **lineplot_kwargs
)



In [ ]:
from carps.analysis.performance_over_time import get_order_by_mean

figure_filename = figpath / "performance_over_time_trials_per_task.pdf"
x="n_trials"
y="trial_value__cost_inc"
hue="optimizer_id"
figsize=None
setup_seaborn(font_scale=1.5)
sorter = get_order_by_mean(df=df, budget_var=x)
df = df.sort_values(by="optimizer_id", key=lambda column: column.map(lambda e: sorter.index(e)))
palette = get_color_palette(df)
grid = sns.FacetGrid(df, col="problem.fid", col_wrap=4, height=4, aspect=1.5, sharex=True, sharey=False)
grid.map_dataframe(sns.lineplot, x=x, y=y, hue=hue, palette=palette, **lineplot_kwargs)
grid.add_legend()
savefig(fig, figure_filename)